# Pause, Resume, and Suspend

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [ ]:
%matplotlib notebook
%run startup.py

# Set up simulated hardware.
from ophyd.sim import SynGauss, SynAxis
motor = SynAxis(name='motor')
det = SynGauss('det', motor, 'motor', center=0, Imax=1,
               noise='uniform', sigma=1, noise_multiplier=0.1)

## Data Acquisition

### Interactively Pausing

In [ ]:
# counts forever until interrupted -- press 'stop' button in the notebook (in the terminal, you would use Ctrl+C)
RE(count([det], delay=1, num=None))

The RunEngine is paused! Our choices:

|Command    |    Outcome|
|-----|-----|
|`RE.resume()`   |Safely resume plan.|
|`RE.abort()`    |Perform cleanup. Mark as aborted.|
|`RE.stop()`     |Perform cleanup. Mark as success.|
|`RE.halt()`     |Do not perform cleanup --- just stop.|
|`RE.state`      |Check if 'paused' or 'idle'.|

While the RunEngine is paused, you have control of the prompt. You can check on things.

In [ ]:
1 + 1

In [ ]:
det.read()

### Safely Resuming

In [ ]:
# Again, we need to press 'stop' or this will run forever....
RE.resume()

### Exit options: abort, stop, and halt

Paused again. Once more, our choices:

|Command    |    Outcome|
|-----|-----|
|`RE.resume()`   |Safely resume plan.|
|`RE.abort()`    |Perform cleanup. Mark as aborted.|
|`RE.stop()`     |Perform cleanup. Mark as success.|
|`RE.halt()`     |Do not perform cleanup --- just stop.|
|`RE.state`      |Check if 'paused' or 'idle'.|

In [ ]:
RE.abort()

Compare 'abort' to 'stop' and 'halt':

In [ ]:
RE(count([det], delay=1, num=None))

In [ ]:
RE.stop()

In [ ]:
RE(count([det], delay=1, num=None))

In [ ]:
RE.halt()

### What happens if we try to run a new plan from a paused state?

In [ ]:
RE(count([det], delay=1, num=None))

In [ ]:
RE(count([det], delay=1, num=None))

In [ ]:
RE.state

In [ ]:
RE.abort()

In [ ]:
RE.state

### Configure a "suspender"

Suspenders are agents set up in advance to run in the background and pause/resume a scan in response to some condition (like a beam dump).

In [ ]:
from bluesky.suspenders import SuspendFloor
import ophyd

current = ophyd.Signal(name='beam_current')

sus = SuspendFloor(current, 50, sleep=1)
RE.install_suspender(sus)

In [ ]:
##### Simulate a beam dump and recovery after 3 seconds....
import threading
import time

def delay_dump_delay_recover():
    time.sleep(3)
    current.set(0)
    time.sleep(5)
    current.set(100)

current.set(100)
thread = threading.Thread(target=delay_dump_delay_recover)
thread.start()
#####

RE(count([det], num=10, delay=1))

## Exercises

1. Execute a `count` with a delay (as we have done many times above) and practice pausing and resuming.
2. Revisit the examples of abort, stop, and halt above.